# 5.3.2
Fine-tuning
Another widely used technique for model reuse, complementary to feature
extraction, is fine-tuning (see figure 5.19). Fine-tuning consists of unfreezing a few of
the top layers of a frozen model base used for feature extraction, and jointly training
both the newly added part of the model (in this case, the fully connected classifier)
and these top layers. This is called fine-tuning because it slightly adjusts the more
abstract representations of the model being reused, in order to make them more rele-
vant for the problem at hand.

I stated earlier that it’s necessary to freeze the convolution base of VGG16 in order to
be able to train a randomly initialized classifier on top. For the same reason, it’s only
possible to fine-tune the top layers of the convolutional base once the classifier on top
has already been trained. If the classifier isn’t already trained, then the error signal
propagating through the network during training will be too large, and the represen-
tations previously learned by the layers being fine-tuned will be destroyed. Thus the
steps for fine-tuning a network are as follow:
    1.  Add your custom network on top of an already-trained base network.
    2.  Freeze the base network.
        Listing 5.22 Freezing all layers up to a specific one
        >>>conv_base.trainable = True
        >>>set_trainable = False
        >>>for layer in conv_base.layers:
        >>>    if layer.name == 'block5_conv1':
        >>>        set_trainable = True
        >>>    if set_trainable:
        >>>        layer.trainable = True
        >>>    else:
        >>>        layer.trainable = False
    3.  Train the part you added.
    4.  Unfreeze some layers in the base network.
    5.  Jointly train both these layers and the part you added.
You already completed the first three steps when doing feature extraction. Let’s pro-
ceed with step 4: you’ll unfreeze your conv_base and then freeze individual layers
inside it.
# As a reminder, this is what your convolutional base looks like:
# >>> conv_base.summary()

You’ll fine-tune the last three convolutional layers, which means all layers up to
block4_pool should be frozen, and the layers block5_conv1, block5_conv2, and
block5_conv3 should be trainable.
Why not fine-tune more layers? Why not fine-tune the entire convolutional base?
You could. But you need to consider the following:
 Earlier layers in the convolutional base encode more-generic, reusable features,
whereas layers higher up encode more-specialized features. It’s more useful to
fine-tune the more specialized features, because these are the ones that need to
be repurposed on your new problem. There would be fast-decreasing returns in
fine-tuning lower layers.
 The more parameters you’re training, the more you’re at risk of overfitting.
The convolutional base has 15 million parameters, so it would be risky to
attempt to train it on your small dataset.
Thus, in this situation, it’s a good strategy to fine-tune only the top two or three layers
in the convolutional base. Let’s set this up, starting from where you left off in the pre-
vious example.

# >>>model.compile(loss='binary_crossentropy',
# >>>optimizer=optimizers.RMSprop(lr=1e-5),
# >>>metrics=['acc'])
# >>>history = model.fit_generator(
# >>>train_generator,
# >>>steps_per_epoch=100,
# >>>epochs=100,
# >>>validation_data=validation_generator,
# >>>validation_steps=50)